In [1]:
!git clone https://github.com/antonzub99/video-coloring.git

Cloning into 'video-coloring'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 105 (delta 45), reused 81 (delta 27), pack-reused 0
Receiving objects: 100% (105/105), 148.72 KiB | 2.92 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [2]:
!mv video-coloring/* .

# 3D-convolutions for temporary consistency: 2-layer head

## Data preparation

We select 5 last videos fram DAVIS dataset and a fragment from Andrey Rublev black-and-white film.

In [3]:
!wget https://data.vision.ee.ethz.ch/csergi/share/davis/DAVIS-2017-trainval-480p.zip
!unzip -d ./videos ./DAVIS-2017-trainval-480p.zip

Streaming output truncated to the last 5000 lines.
  inflating: ./videos/DAVIS/JPEGImages/480p/color-run/00078.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/color-run/00079.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/color-run/00080.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/color-run/00081.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/color-run/00082.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/color-run/00083.jpg  
   creating: ./videos/DAVIS/JPEGImages/480p/cows/
  inflating: ./videos/DAVIS/JPEGImages/480p/cows/00000.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/cows/00001.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/cows/00002.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/cows/00003.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/cows/00004.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/cows/00005.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/cows/00006.jpg  
  inflating: ./videos/DAVIS/JPEGImages/480p/cows/00007.jpg  
  inflating: ./

In [4]:
!gdown 1UObl9IPKHLZcKSxK7Lhl4QGmxra6rbLg

Downloading...
From: https://drive.google.com/uc?id=1UObl9IPKHLZcKSxK7Lhl4QGmxra6rbLg
To: /content/val.zip
100% 5.59M/5.59M [00:00<00:00, 38.4MB/s]


In [5]:
!unzip -d ./rublev val.zip

Archive:  val.zip
  inflating: ./rublev/img001.jpg     
  inflating: ./rublev/img002.jpg     
  inflating: ./rublev/img003.jpg     
  inflating: ./rublev/img004.jpg     
  inflating: ./rublev/img005.jpg     
  inflating: ./rublev/img006.jpg     
  inflating: ./rublev/img007.jpg     
  inflating: ./rublev/img008.jpg     
  inflating: ./rublev/img009.jpg     
  inflating: ./rublev/img010.jpg     
  inflating: ./rublev/img011.jpg     
  inflating: ./rublev/img012.jpg     
  inflating: ./rublev/img013.jpg     
  inflating: ./rublev/img014.jpg     
  inflating: ./rublev/img015.jpg     
  inflating: ./rublev/img016.jpg     
  inflating: ./rublev/img017.jpg     
  inflating: ./rublev/img018.jpg     
  inflating: ./rublev/img019.jpg     
  inflating: ./rublev/img020.jpg     
  inflating: ./rublev/img021.jpg     
  inflating: ./rublev/img022.jpg     
  inflating: ./rublev/img023.jpg     
  inflating: ./rublev/img024.jpg     
  inflating: ./rublev/img025.jpg     
  inflating: ./rublev/img026.jpg

In [6]:
import dataset 

In [7]:
dawis_dataset = dataset.VideoDataset('./videos/DAVIS/JPEGImages/480p', frame_stack=1, img_size=128, need_full_videos=True)

100%|██████████| 90/90 [01:43<00:00,  1.15s/it]


In [8]:
rublev = dataset.OneBWVideoDataset('./rublev', frame_stack=5, img_size=128)

In [9]:
from torch.utils.data import DataLoader

rublev_dataset = DataLoader(rublev, num_workers=2, batch_size=32, shuffle=False)

## Define Conv3D model and train loop

In [10]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 584 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 60.8 MB/s 
     |████████████████████████████████| 409 kB 59.2 MB/s 
     |████████████████████████████████| 140 kB 77.0 MB/s 
     |████████████████████████████████| 1.1 MB 37.7 MB/s 
     |████████████████████████████████| 144 kB 74.9 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 20.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
!pip install wandb
!wandb login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 66.7 MB/s 
     |████████████████████████████████| 145 kB 63.6 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=f0bb252d3c8b7c4c8556ec66b1d889a90b47f98c5c83ceb354dd9695d5c69837
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
from conv3d_model import ColorNet3D_2layer

In [13]:
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import models
import numpy as np
from scipy import linalg

import losses



class TrainLoss(nn.Module):
    """
        Loss calculator
    """
    def __init__(self):
        super().__init__()

    def forward(self, pred_imgs, real_imgs):
        return F.mse_loss(pred_imgs, real_imgs)


class ValLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.lpips = losses.PerceptualLoss()
    
    def forward(self, pred_imgs, real_imgs):
        return ( 
            F.mse_loss(pred_imgs, real_imgs), 
            self.lpips(pred_imgs, real_imgs),
            losses.psnr(pred_imgs, real_imgs)
        )


In [14]:
import aug
train_dataset = dataset.VideoDataset('./videos/DAVIS/JPEGImages/480p', 5, 
                                     used_classes=list(range(85)), 
                                     transform=aug.transform_train,
                                     img_size=128)
test_dataset = dataset.VideoDataset('./videos/DAVIS/JPEGImages/480p', 5, 
                                    used_classes=list(range(85,90)), 
                                    transform=aug.transform_test,
                                    img_size=128)

100%|██████████| 90/90 [00:05<00:00, 17.39it/s]


In [15]:
# Copyright The PyTorch Lightning team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Modifications Copyright Skoltech Deep Learning Course.

import torch
import wandb
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from skimage import color


class ColorModel(pl.LightningModule):
    def __init__(
        self,
        model: str,
        backbone: str,
        temp_conv_size: int = 3,
        optimizer: str = 'default',
        scheduler: str = 'default',
        lr: float = None,
        wd : float = None,
        batch_size: int = 16
    ):
        super().__init__()
        self.net = ColorNet3D_2layer(backbone)

        self.train_dataset = train_dataset
        self.test_dataset = test_dataset

        self.train_loss = TrainLoss()
        self.val_loss = ValLoss()

        self.batch_size = batch_size
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.lr = lr
        self.wd = wd
        self.eps = 1e-7

    def forward(self, x):
        return self.net(x.float()).float()

    def training_step(self, batch, batch_idx):
        lum, ab  = batch
        ab_pred = self.forward(lum)
        train_loss = self.train_loss(ab_pred, ab.float())

        self.log('train_loss', train_loss, prog_bar=True)

        return train_loss

    def validation_step(self, batch, batch_idx):
        lum, ab = batch
        pred_ab = self.forward(lum)

        result = dict()

        lab_preds = torch.cat((lum, pred_ab), dim=1).permute(0,2,3,4,1).cpu().numpy()
        lab_reals = torch.cat((lum, ab), dim=1).permute(0,2,3,4,1).cpu().numpy()

        img_pred = torch.tensor(color.lab2rgb(lab_preds)).permute(0, 4, 1, 2, 3).to(ab.device)
        img_real =  torch.tensor(color.lab2rgb(lab_reals)).permute(0, 4, 1, 2, 3).to(ab.device)


        if batch_idx == 0:
            result.update({
                'lum': lum[0].repeat(3,1,1,1) / 100,
                'img_pred': img_pred[0], 
                'img_real': img_real[0]
            })

        l1_ab = self.train_loss(pred_ab, ab)

        B, C, T, H, W = img_pred.shape
        preds_to_lpips = img_pred.permute(0, 2, 1, 3, 4).reshape(-1, C, H, W)
        reals_to_lpips = img_real.permute(0, 2, 1, 3, 4).reshape(-1, C, H, W)
        
        l1_rgb, lpips, psnr = self.val_loss(preds_to_lpips.float(), reals_to_lpips.float())
        result.update({
            'test_loss': l1_ab, 
            'l1_rgb': l1_rgb, 
            'lpips': lpips,
            'psnr' : psnr
        })

        return result

    def validation_epoch_end(self, outputs):
        test_loss = np.mean([x['test_loss'].cpu().item() for x in outputs])
        l1_rgb = np.mean([x['l1_rgb'].cpu().item() for x in outputs])
        lpips = np.mean([x['lpips'].cpu().item() for x in outputs])
        psnr = np.mean([x['psnr'].cpu().item() for x in outputs])

        log_dict = {'test_loss': test_loss, 'mse_rgb': l1_rgb, 'lpips': lpips, 'psnr': psnr}

        for k, v in log_dict.items():
            self.log(k, v, prog_bar=True)

        # Visualize results
        lum = outputs[0]['lum'].cpu().permute(1,0,2,3)
        img_pred = outputs[0]['img_pred'].cpu().permute(1,0,2,3)
        img_real = outputs[0]['img_real'].cpu().permute(1,0,2,3)

        results = torch.cat(torch.cat([lum, img_pred, img_real], dim=3).split(1, dim=0), dim=2)
        results_thumbnail = F.interpolate(results, scale_factor=0.9, mode='bilinear')[0]

        self.logger.log_image('results', [results_thumbnail], self.current_epoch)

    def configure_optimizers(self):
        opt = torch.optim.Adam(params=self.net.parameters(), lr=self.lr, weight_decay=self.wd)
        #sch = torch.optim.lr_scheduler.StepLR(opt, step_size=5, gamma=0.5)
        return [opt]#[sch]

    def train_dataloader(self):
        return DataLoader(self.train_dataset, num_workers=2, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_dataset, num_workers=2, batch_size=self.batch_size, shuffle=False)

In [24]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

def train(model, experiment_name, epochs, use_gpu):
    assignment_dir = './colorization'

    logger = WandbLogger(project='colorization_baseline', save_dir=f'{assignment_dir}/logs', name=experiment_name)
    #logger.watch(model, log="all")
    
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        monitor='psnr',
        dirpath=f'{assignment_dir}/logs/{experiment_name}',
        filename='{epoch:02d}',
        every_n_epochs=10,
        mode='max')

    trainer = pl.Trainer(
        max_epochs=epochs, 
        gpus=1 if use_gpu else None, 
        benchmark=True, 
        logger=logger,
        log_every_n_steps=10,
        check_val_every_n_epoch=1,
        callbacks=[checkpoint_callback],
        ) 
    
    trainer.fit(model)
    wandb.finish()

In [17]:
model = ColorModel('deeplab', 'color', batch_size=8, lr=1e-5, wd=1e-4)

Downloading: "https://colorizers.s3.us-east-2.amazonaws.com/colorization_release_v2-9b330a0b.pth" to /root/.cache/torch/hub/checkpoints/colorization_release_v2-9b330a0b.pth


  0%|          | 0.00/123M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
train(model, 'CIC-DeepLab 2-layer head finetuning + aug, lr=1e-5 wd=1e-4', use_gpu=True)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

wandb: Currently logged in as: dtiapkin. Use `wandb login --relogin` to force relogin
wandb: WARNING Path ./colorization/logs/wandb/ wasn't writable, using system temp directory.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | net        | ColorNet3D_2layer | 34.9 M
1 | train_loss | TrainLoss         | 0     
2 | val_loss   | ValLoss           | 12.9 M
-------------------------------------------------
2.6 M     Trainable params
45.2 M    Non-trainable params
47.8 M    Total params
191.312   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 313 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 73 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 4210 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lpips,█▄▃▃▄▂▃▂▄▃▃▃▁▂▂▃▁▂▁▁▂▄▂▃▂▂▂▁▁▃
mse_rgb,█▃▃▃▃▂▃▁▄▃▂▃▁▂▂▂▁▂▁▁▂▄▂▃▂▂▂▁▁▂
psnr,▁▄▆▅▅▇▅▇▄▅▆▅█▆▆▆▇▇██▆▅▇▆▇▆▇█▇▆
test_loss,█▃▂▂▃▁▃▁▃▃▂▂▁▂▂▂▁▂▁▁▂▃▁▂▂▂▁▁▁▂
train_loss,▅▃▃▂█▃▆▅▂▁▆▅▃▂▃▂█▅▂▃▂▃▃▅▄▃▁▁▃▄▂▂▁▅▂▂▂▃▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▁▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,29
lpips,0.51924
mse_rgb,0.00921
psnr,16.62072


In [21]:
PATH = "conv3d_2layer_wo_ft.ckpt"
#torch.save(model.state_dict(), PATH)
model = ColorModel('deeplab', 'color', batch_size=8, lr=1e-6, wd=1e-4)
model.load_state_dict(torch.load(PATH))
#model = ColorModel.load_from_checkpoint(PATH, model='deeplab', backbone='color', batch_size=8, lr=1e-6, wd=1e-4)

<All keys matched successfully>

In [22]:
#model.lr = 1e-6
for param in model.net.features.parameters():
    param.requires_grad = True

In [25]:
train(model, 'CIC-DeepLab 2-layer head, +finetuning + aug, lr=1e-5 wd=1e-4', epochs=10, use_gpu=True)

wandb: Currently logged in as: dtiapkin. Use `wandb login --relogin` to force relogin
wandb: WARNING Path ./colorization/logs/wandb/ wasn't writable, using system temp directory.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | net        | ColorNet3D_2layer | 34.9 M
1 | train_loss | TrainLoss         | 0     
2 | val_loss   | ValLoss           | 12.9 M
-------------------------------------------------
34.9 M    Trainable params
12.9 M    Non-trainable params
47.8 M    Total params
191.312   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 63 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 4770 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 7 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 198 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
lpips,█▄▄▄▅▁▁▄▁▂
mse_rgb,█▃▃▃▄▁▁▄▁▂
psnr,▁▆▅▅▄██▄█▇
test_loss,█▃▃▃▄▁▁▅▁▂
train_loss,▅▂▃▂▄▄▃▅▆▆▅▅▄▇▂▅▃▁█▂▂▃▄▃▂▁▃▃▄▂▄▇▂▃▃▄▁▂▃▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▁▇▇▇▁▇███
epoch,9
lpips,0.46219
mse_rgb,0.00653
psnr,17.94659


In [26]:
PATH = "conv3d_2layer_w_ft.ckpt"
torch.save(model.state_dict(), PATH)

### Validation

In [27]:
model = model.cuda().eval()

In [28]:
import matplotlib.pyplot as plt
import imageio
import cv2
import torch.nn.functional as F
import torch
import numpy as np
from skimage import color

for i in range(85, 90):
    with torch.no_grad():
        # C x T x H x W
        lum, ab = dawis_dataset[i]
        # C x T x H x W
        pred_ab = model(lum[None,...].float().cuda()).cpu()[0]

        lab_preds = torch.cat((lum, pred_ab), dim=0).permute(1,2,3,0).numpy()
        lab_reals = torch.cat((lum, ab), dim=0).permute(1,2,3,0).numpy()

        img_pred = torch.tensor(color.lab2rgb(lab_preds))
        img_real =  torch.tensor(color.lab2rgb(lab_reals))

        video = np.concatenate((lum.repeat(3,1,1,1).permute(1,2,3,0) / 100, img_pred, img_real), axis=2)
    

    gif_name = 'DAVIS_{}.gif'.format(i)
    imageio.mimsave(gif_name, (np.clip(video, 0, 1) * 255).astype(np.uint8))

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 189 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


Final videos:

<img src="https://media.giphy.com/media/7cYMclrRDoiioAYsNe/giphy.gif">


<img src="https://media.giphy.com/media/I3szDlnE2UsU6s6d5c/giphy.gif">


<img src="https://media.giphy.com/media/xILvQyrIzxJ4gi5l4g/giphy.gif">


<img src="https://media.giphy.com/media/R440koCYWZdfKZ4uYe/giphy.gif">


<img src="https://media.giphy.com/media/EpDdCaZk5GyA6daw9q/giphy.gif">


### Run on RublevNet dataset

In [29]:
rublev_video = []

with torch.no_grad():
    for batch in rublev_dataset:
        lum = batch.float().cuda()
        ab = model(lum).cpu()

        lab = torch.cat((lum.cpu(), ab), dim=1).permute(0,2,3,4,1).flatten(end_dim=1)
        rgb = color.lab2rgb(lab)
        wb = lum.cpu().repeat(1,3,1,1,1).permute(0,2,3,4,1).flatten(end_dim=1).numpy()

        rublev_video.append(np.concatenate((wb / 100, rgb), axis=2))


/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 16 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


In [30]:
final_rublev = np.concatenate(rublev_video, axis=0)

In [31]:
final_rublev.shape

(375, 128, 256, 3)

In [32]:
imageio.mimsave('rublev.gif', (np.clip(final_rublev, 0, 1) * 255).astype(np.uint8))

Result:

<img width=512 src="https://media.giphy.com/media/TST300PTDtx62WjOiS/giphy-downsized-large.gif">